In [ ]:
import numpy as np
import re
from collections import Counter

# Limpando os dados

In [104]:
with open("corpus_brasileiro.txt", encoding='ISO-8859-1') as f:
    lines = f.readlines()

In [105]:
def limpar_palavra(a):
    """
    Input: palavra
    Output: palavra sem maiusculas, simbulos, numeros our acentos
    """
    a = a.lower()
    a = re.sub('\W+','', a)
    a = re.sub(r'[~^0-9]', '', a)
    a = a.replace("'","")
    a = a.replace(".","")
    a = a.replace(",","")
    a = a.replace("ã","a")
    a = a.replace("á","a")
    a = a.replace("ä","a")
    a = a.replace("â","a")
    a = a.replace("à","a")
    a = a.replace("é","e")
    a = a.replace("ê","e")
    a = a.replace("õ","o")
    a = a.replace("ó","o")
    a = a.replace("ó","o")
    a = a.replace("ç","c")
    a = a.replace("ú","u")
    a = a.replace("í","i")
    a = a.replace("ü","u")
    a = a.replace("ñ","n")
    return a

In [106]:
# Vamos transformar o corpus em um dicionário que só contem palavras limpas de 5 letras 
corpus = dict()
for line in lines:
    # Seperando as linhas 
    split_line = line.split('\t')
    frequencia = int(split_line[0])
    palavra = split_line[1].split('\n')[0]
    # Limpando as palavras
    palavra_limpa = limpar_palavra(palavra)
    # Selecionando palavras com 5 letras apenas
    if len(palavra_limpa) !=5:
        continue
    # Somando as frequencias de palavras identicas apos a limpeza
    if palavra_limpa in corpus.keys():
        velha_frequencia = corpus[palavra_limpa]
        corpus[palavra_limpa] = velha_frequencia + frequencia
    else:
        corpus[palavra_limpa] = frequencia

In [62]:
# Vamos restringir o corpus a palavaras que só aparecem com alta frequencia
corpus_limpo = {k:v for (k,v) in corpus.items() if v >= 500} 
palavras_muito_usadas = {k:v for (k,v) in corpus.items() if v > 5000} 

In [64]:
print(len(corpus))
print(len(palavras_muito_usadas))
print(len(corpus_limpo))

165862
1544
5350


In [71]:
def mostrar_n_palavras_mais_frequentes(corp,n):
    # Recebe um dicionário e mostra as n palavras com maior frequencua
    corp_ordem = sorted(corp, key=corp.get,reverse=True)
    print(corp_ordem[:n])

In [70]:
mostrar_n_palavras_mais_frequentes(corpus_limpo, 40)

['entre', 'sobre', 'foram', 'paulo', 'mesmo', 'ainda', 'muito', 'maior', 'forma', 'final', 'todos', 'tempo', 'assim', 'sendo', 'parte', 'saude', 'disse', 'grupo', 'ontem', 'outro', 'local', 'pelos', 'folha', 'fazer', 'dados', 'menos', 'mundo', 'podem', 'desde', 'antes', 'total', 'poder', 'entao', 'valor', 'todas', 'media', 'deste', 'agora', 'detaq', 'vezes']


# Funções do jogo

In [107]:
def chute_chave_sinal(chute,chave):
    """
    Input: duas strings, chute e chave
    Output: sinal uma lista de tuples onde a primeira entrada de cada tupla é uma letra e a segunda um 
            sinal: 0 - letra não se encontra na palavra
                   1 - letra na palavra em outra posicão
                   2 - letra na palavra naquela posição
    """
    sinal = list((l, 0) for l in chute)
    pos_pular = list()
    # Verificando verdes
    for pos in range(len(chute)):
        if chute[pos] == chave[pos]:
            sinal[pos] = (chute[pos],2)
            pos_pular.append(pos)
    # Verificando amarelos
    for pos in range(len(chute)):
        if pos in pos_pular:
            continue
        letra_chute = chute[pos]
        for chave_pos in range(len(chave)):
            if chave_pos in pos_pular:
                continue
            if letra_chute == chave[chave_pos]:
                sinal[pos] = (letra_chute,1)   
                pos_pular.append(chave_pos)
    return sinal 

In [108]:
def palavra_respeita_sinal(sinal, palavra):
    """
    Inputs: sinal (lista) e uma palavra
    Outputs: boolean dizendo se a palavra está de acordo com o sinal
    """
    pos_pular = list()
    # Resolvendo sinal 2
    for pos in range(len(sinal)):
        letra_sinal = sinal[pos][0]
        codigo_sinal = sinal[pos][1]
        if codigo_sinal != 2 and letra_sinal == palavra[pos]:
            return False
        if codigo_sinal == 2:
            if letra_sinal != palavra[pos]: 
                return False
            else:
                pos_pular.append(pos)
    # Verificando o resto
    for pos in range(len(sinal)):
        if pos in pos_pular:
            continue
        letra_sinal = sinal[pos][0]
        codigo_sinal = sinal[pos][1]
        # Primeiro letras que não fazem parte da resposta, sinal 0
        if codigo_sinal == 0:   
            for pal_pos in range(len(palavra)):
                if pal_pos in pos_pular:
                    continue
                if letra_sinal == palavra[pal_pos]:
                    return False
        # signal code == 1
        if codigo_sinal == 1: 
            letra_encontrada = False
            for pal_pos in range(len(palavra)):
                if pal_pos in pos_pular:
                    continue
                if letra_sinal == palavra[pal_pos]:
                    letra_encontrada = True
            if not letra_encontrada:
                return False
    return True     

In [109]:
def sinal_corpus(sinal, corpo):
    """
    Inputs: Sinal (lista) e um corpo (dicionario de palavras)
    Outpt: Novo corpo que respeita o sinal
    """
    corpo_novo  = {k:v for (k,v) in corpo.items() if palavra_respeita_sinal(sinal,k)}    
    return corpo_novo

In [110]:
def ganho_entropia(corp1, corp2):
    """
    Computa o ganho de entropia de se ir do corpo 1 para o corpo 2
    """
    return np.log2((len(corp1)+1)/(len(corp2)+1))

In [111]:
def ganho_entropia_esperado(chute, corpo_candidato):
    """
    Inputs: a guess (string), a candidate corp
    Outputs: Expected entropy Gain    
    """
    frequencia_total = 0
    entropia_esperada = 0
    for chave, frequencia in corpo_candidato.items():
        # frequencia = np.log2(frequencia)
        frequencia_total += frequencia
        novo_sinal = chute_chave_sinal(chute,chave)
        corpo_novo = sinal_corpus(novo_sinal, corpo_candidato)
        entropia = ganho_entropia(corpo_candidato, corpo_novo)
        entropia_esperada += frequencia*entropia
    entropia_esperada /= frequencia_total      
    return entropia_esperada

In [135]:
def max_ganho_entropia_esperado(corpo_chutes,corpo_candidatos):
    """
    Olha todas as palavras em um corpo de chutes e escolhe aquela que maximiza o ganho de entropia esperado no corpo de candidatos
    Input: 2 corpos com dicionario de frequencias de palavras
           corpo de chutes terá as palavras que poderão ser chutes
           corpo de candidatos são as palavras que ainda podem ser respostas
           Ambos os corpos podems ser iguais
    Outputs: Uma tupla com a maior entropia esperada
    """
    melhor_palavra = None
    max_e_entropia = -np.inf
    for chute, frequency in corpo_chutes.items():
        e_entropia = expected_entropy_gain(chute, corpo_candidatos)
        if e_entropia > max_e_entropia:
            melhor_palavra = chute
            max_e_entropia = e_entropia
            print('Melhor chute = {}, Entropia esperada = {}'.format(melhor_palavra,max_e_entropia))
    return melhor_palavra, max_e_entropia

# Melhor chute inicial

In [119]:
# max_expected_entropy_guess(corpus_limpo ,palavras_muito_usadas)

In [120]:
max_ganho_entropia_esperado(palavras_muito_usadas,palavras_muito_usadas)

Melhor chute = agudo, Entropia esperada = 4.495127383518988
Melhor chute = havia, Entropia esperada = 5.4117142382414904
Melhor chute = timar, Entropia esperada = 5.477417673969076
Melhor chute = salas, Entropia esperada = 6.518109370342865
Melhor chute = ossos, Entropia esperada = 6.820929832379818
Melhor chute = atlas, Entropia esperada = 6.941371834334802
Melhor chute = adams, Entropia esperada = 7.469284792748463
Melhor chute = amaro, Entropia esperada = 7.806653716093225


('amaro', 7.806653716093225)

In [121]:
ganho_entropia_esperado('asaro', palavras_muito_usadas)

7.839635145944346

# Jogando um jogo

### Jogando um jogo simulado
* A palavra correta era "sogro"
* Eu comecei com o melhor chute: "amaro"
* Depois usei o melhor chute apesar dele não estar no corpo de respostas que satisfaz o primeiro sinal: "astor"
* Na terceira dica eu comecei a ir na resposta mais provável: "sopro". 
* Finalmente só sobrou a resposta correta: "sogro".


In [128]:
sinal_1 = [('a', 0), ('m', 0), ('a', 0), ('r', 2), ('o', 2)]

In [130]:
corpus_sinal_1 = signal_corpus(sinal_1, corpus_limpo)
print(len(corpus_sinal_1))
print(mostrar_n_palavras_mais_frequentes(corpus_sinal_1,40))
max_ganho_entropia_esperado(corpus_limpo,corpus_sinal_1)

40
['outro', 'livro', 'pedro', 'quero', 'lucro', 'negro', 'ferro', 'vitro', 'vidro', 'dobro', 'couro', 'utero', 'litro', 'genro', 'choro', 'tupro', 'forro', 'touro', 'sopro', 'cloro', 'clero', 'burro', 'cerro', 'louro', 'sogro', 'cedro', 'entro', 'piero', 'otero', 'libro', 'douro', 'curro', 'loiro', 'cipro', 'zorro', 'serro', 'pdtro', 'corro', 'cobro', 'nigro']
None
Melhor chute = agudo, Entropia esperada = 2.0975744525206483
Melhor chute = virol, Entropia esperada = 5.214476414496481
Melhor chute = astor, Entropia esperada = 5.357552004618084


('astor', 5.357552004618084)

In [133]:
sinal_2 = [('a', 0), ('s', 1), ('t', 0), ('o', 1), ('r', 1)]

In [139]:
corpus_sinal_2 = signal_corpus(sinal_2, corpus_sinal_1)
print(len(corpus_sinal_2))
print(mostrar_n_palavras_mais_frequentes(corpus_sinal_2,40))
max_ganho_entropia_esperado(corpus_sinal_2,corpus_sinal_2)

3
['sopro', 'sogro', 'serro']
None
Melhor chute = sopro, Entropia esperada = 1.0


('sopro', 1.0)

In [137]:
sinal_3 = [('s', 2), ('o', 2), ('p', 0), ('r', 2), ('o', 2)]

In [138]:
corpus_sinal_3 = signal_corpus(sinal_3, corpus_sinal_2)
print(len(corpus_sinal_3))
print(mostrar_n_palavras_mais_frequentes(corpus_sinal_3,40))
max_ganho_entropia_esperado(corpus_sinal_3,corpus_sinal_3)

1
['sogro']
None
Melhor chute = sogro, Entropia esperada = 0.0


('sogro', 0.0)

## Rascunho